# **Tarea con Polars**

**Segundo Ejercicio:**Lo desarrollé teniendo presentev lo expuesto en el tablero.

**Algoritmo Apriori**

In [2]:
# Configuración pensando en términos de probabilidades.

conjuntos = [
    {'f','a','c','d','g','i','m','p'},
    {'a','b','c','f','l','m','o'},
    {'b','f','h','j','o'},
    {'b','c','k','s','p'},
    {'a','f','c','e','l','p','m','n'}
]

In [4]:
from itertools import combinations
from collections import defaultdict



min_support = 2
min_confidence = 0.5  # 50%

# ------------------------------------------
# 1. Algoritmo Apriori (Modificado para guardar soportes y tener presente probabilidades)
# ------------------------------------------
def apriori_with_support(datos, soporte_min):
    support_dict = {}
    # Paso 1: Itemsets de tamaño 1
    conteo = defaultdict(int)
    for trans in datos:
        for item in trans:
            key = frozenset([item])
            conteo[key] += 1
    support_dict.update(conteo)
    frecuentes = [[k for k, v in conteo.items() if v >= soporte_min]]

    k = 1
    while True:
        # Generar candidatos
        items = list({i for s in frecuentes[-1] for i in s})
        candidatos = [frozenset(c) for c in combinations(items, k+1)]

        # Podar y calcular soporte
        conteo_candidatos = defaultdict(int)
        for trans in datos:
            for cand in candidatos:
                if cand.issubset(trans):
                    conteo_candidatos[cand] += 1
        support_dict.update(conteo_candidatos)

        nuevos_frec = [k for k, v in conteo_candidatos.items() if v >= soporte_min]
        if not nuevos_frec:
            break
        frecuentes.append(nuevos_frec)
        k += 1

    return support_dict

# 2. Ejecutar Apriori
apriori_support = apriori_with_support(conjuntos, min_support)

# ------------------------------------------
# 3. Función común para generar reglas
# ------------------------------------------
def generar_reglas(support_dict, min_conf):
    reglas = []
    itemsets = [k for k in support_dict.keys() if len(k) > 1]

    for itemset in itemsets:
        items = list(itemset)
        # Generar todos los posibles antecedentes
        for i in range(1, len(items)):
            for antecedente in combinations(items, i):
                antecedente = frozenset(antecedente)
                consecuente = itemset - antecedente
                soporte_total = support_dict[itemset]
                soporte_antecedente = support_dict.get(antecedente, 0)

                if soporte_antecedente == 0:
                    continue

                confianza = soporte_total / soporte_antecedente
                if confianza >= min_conf:
                    reglas.append({
                        'regla': f"{set(antecedente)} => {set(consecuente)}",
                        'soporte': soporte_total,
                        'confianza': round(confianza, 2)
                    })
    return reglas

# Generar reglas para ambos algoritmos
reglas_apriori = generar_reglas(apriori_support, min_confidence)

# 4. Resultados
# ------------------------------------------
print("Reglas de Apriori (Confianza ≥ 50%):")
for regla in reglas_apriori:
    print(f"{regla['regla']} | Soporte: {regla['soporte']}, Confianza: {regla['confianza']}")


Reglas de Apriori (Confianza ≥ 50%):
{'f'} => {'a'} | Soporte: 3, Confianza: 0.75
{'a'} => {'f'} | Soporte: 3, Confianza: 1.0
{'a'} => {'c'} | Soporte: 3, Confianza: 1.0
{'c'} => {'a'} | Soporte: 3, Confianza: 0.75
{'p'} => {'a'} | Soporte: 2, Confianza: 0.67
{'a'} => {'p'} | Soporte: 2, Confianza: 0.67
{'a'} => {'m'} | Soporte: 3, Confianza: 1.0
{'m'} => {'a'} | Soporte: 3, Confianza: 1.0
{'f'} => {'c'} | Soporte: 3, Confianza: 0.75
{'c'} => {'f'} | Soporte: 3, Confianza: 0.75
{'p'} => {'f'} | Soporte: 2, Confianza: 0.67
{'f'} => {'p'} | Soporte: 2, Confianza: 0.5
{'f'} => {'m'} | Soporte: 3, Confianza: 0.75
{'m'} => {'f'} | Soporte: 3, Confianza: 1.0
{'p'} => {'c'} | Soporte: 3, Confianza: 1.0
{'c'} => {'p'} | Soporte: 3, Confianza: 0.75
{'m'} => {'c'} | Soporte: 3, Confianza: 1.0
{'c'} => {'m'} | Soporte: 3, Confianza: 0.75
{'p'} => {'m'} | Soporte: 2, Confianza: 0.67
{'m'} => {'p'} | Soporte: 2, Confianza: 0.67
{'l'} => {'b'} | Soporte: 1, Confianza: 0.5
{'b'} => {'f'} | Soporte: 2

**Algoritmo FP-Growth**

In [10]:
from collections import defaultdict
from itertools import combinations


class TreeNode:
    def __init__(self, name, parent):
        self.name = name       # Nombre del ítem
        self.parent = parent   # Nodo padre
        self.children = {}     # Diccionario de hijos {nombre: nodo}
        self.count = 0         # Contador de soporte
        self.link = None       # Enlace a nodos del mismo ítem

def build_fp_tree(conjuntos, min_support):
    # Paso 1: Calcular frecuencia de ítems
    item_counts = defaultdict(int)
    for trans in conjuntos:
        for item in trans:
            item_counts[item] += 1

    # Filtrar ítems frecuentes y ordenar
    freq_items = {item: count for item, count in item_counts.items() if count >= min_support}
    if not freq_items:
        return None, None
    sorted_items = sorted(freq_items.items(), key=lambda x: (-x[1], x[0]))
    order = [item[0] for item in sorted_items]

    # Paso 2: Construir FP-Tree y tabla de cabecera
    root = TreeNode('root', None)
    header_table = {item: [count, None] for item, count in sorted_items}

    for trans in conjuntos:
        # Filtrar y ordenar transacción
        filtered = [item for item in trans if item in freq_items]
        filtered.sort(key=lambda x: (order.index(x), x))
        if not filtered:
            continue

        # Insertar en el árbol
        current_node = root
        for item in filtered:
            if item in current_node.children:
                current_node = current_node.children[item]
                current_node.count += 1
            else:
                new_node = TreeNode(item, current_node)
                new_node.count = 1
                current_node.children[item] = new_node

                # Enlazar a la tabla de cabecera
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    node = header_table[item][1]
                    while node.link is not None:
                        node = node.link
                    node.link = new_node
                current_node = new_node

    return root, header_table

def mine_fp_tree(header_table, min_support, prefix, freq_itemsets):
    items = list(header_table.items())
    # Procesar desde el ítem menos frecuente
    for item, (count, node) in sorted(items, key=lambda x: (x[1][0], x[0])):
        new_prefix = prefix.copy()
        new_prefix.add(item)
        freq_itemsets.append((new_prefix, count))  # Añadir itemset actual

        # Paso 3: Encontrar patrones condicionales
        conditional_paths = []
        current_node = node
        while current_node is not None:
            path = []
            parent = current_node.parent
            while parent.name != 'root':
                path.append(parent.name)
                parent = parent.parent
            path.reverse()
            if path:
                conditional_paths.append((path, current_node.count))
            current_node = current_node.link

        # Construir transacciones condicionales
        conditional_trans = []
        for path, cnt in conditional_paths:
            conditional_trans.extend([path] * cnt)

        # Construir sub-árbol condicional
        sub_root, sub_header = build_fp_tree(conditional_trans, min_support)
        if sub_header is not None:
            mine_fp_tree(sub_header, min_support, new_prefix, freq_itemsets)

def fp_growth(transacciones, min_support):
    freq_itemsets = []
    root, header_table = build_fp_tree(transacciones, min_support)
    if header_table:
        mine_fp_tree(header_table, min_support, set(), freq_itemsets)
    # Filtrar y formatear resultados
    result = defaultdict(int)
    for itemset, count in freq_itemsets:
        key = frozenset(itemset)
        result[key] = max(result[key], count)
    return sorted([(set(k), v) for k, v in result.items()], key=lambda x: (len(x[0]), x[0]))

# Ejecutar con soporte mínimo = 2
resultado = fp_growth(conjuntos, 2)

# Mostrar resultados Básicos de la configuración de los diferentes ítems
print("Itemsets frecuentes con FP-Growth:")
for itemset, soporte in resultado:
    print(f"{set(itemset)}: Soporte = {soporte}")

# Create fp_growth_support dictionary
fp_growth_support = {}
for itemset, support in resultado:
    fp_growth_support[frozenset(itemset)] = support

# ------------------------------------------
# 3. Función común para generar reglas
# ------------------------------------------
def generar_reglas(support_dict, min_conf):
    reglas = []
    itemsets = [k for k in support_dict.keys() if len(k) > 1]

    for itemset in itemsets:
        items = list(itemset)
        # Generar todos los posibles antecedentes
        for i in range(1, len(items)):
            for antecedente in combinations(items, i):
                antecedente = frozenset(antecedente)
                consecuente = itemset - antecedente
                soporte_total = support_dict[itemset]
                soporte_antecedente = support_dict.get(antecedente, 0)

                if soporte_antecedente == 0:
                    continue

                confianza = soporte_total / soporte_antecedente
                if confianza >= min_conf:
                    reglas.append({
                        'regla': f"{set(antecedente)} => {set(consecuente)}",
                        'soporte': soporte_total,
                        'confianza': round(confianza, 2)
                    })
    return reglas

# Generar reglas para ambos algoritmos
reglas_fp_growth = generar_reglas(fp_growth_support, min_confidence)

print("\nReglas de FP-Growth (Confianza ≥ 50%):")
for regla in reglas_fp_growth:
    print(f"{regla['regla']} | Soporte: {regla['soporte']}, Confianza: {regla['confianza']}")

Itemsets frecuentes con FP-Growth:
{'d'}: Soporte = 2
{'a'}: Soporte = 4
{'b'}: Soporte = 4
{'c'}: Soporte = 4
{'d', 'c'}: Soporte = 2
{'b', 'a'}: Soporte = 2
{'a', 'c'}: Soporte = 2
{'b', 'c'}: Soporte = 3

Reglas de FP-Growth (Confianza ≥ 50%):
{'d'} => {'c'} | Soporte: 2, Confianza: 1.0
{'c'} => {'d'} | Soporte: 2, Confianza: 0.5
{'b'} => {'a'} | Soporte: 2, Confianza: 0.5
{'a'} => {'b'} | Soporte: 2, Confianza: 0.5
{'a'} => {'c'} | Soporte: 2, Confianza: 0.5
{'c'} => {'a'} | Soporte: 2, Confianza: 0.5
{'b'} => {'c'} | Soporte: 3, Confianza: 0.75
{'c'} => {'b'} | Soporte: 3, Confianza: 0.75
